# COUNT and COUNT DISTINCT

## Overview

I am a junior data analyst working for a company that manufactures socks. I have access to data on the company’s customers, orders, warehouses, and products. For this activity, I will use COUNT and COUNT DISTINCT to determine the amounts of things to help answer business questions.

## Dataset

I receive 2 .csv files containing the data for orders and warehouses:

- the **Warehouse table** can be viewed in [Google Sheets](https://drive.google.com/file/d/18bzqeHv2Nk_BZD0N8S9WlpC2mSm--SVd/view?usp=drive_link) or the [.csv file](/activities/sql/c05m03-count-distinct/c05m03-warehouse-data.csv), and records are saved in the format: warehouse_id,warehouse_alias,maximum_capacity,employee_total,state
- the **Orders table** can be viewed in [Google Sheets](https://drive.google.com/file/d/1dPcjBa1mC1FFtQsMZ95CnL_NBoVSzEP5/view?usp=drive_link) or the [.csv file](/activities/sql/c05m03-count-distinct/c05m03-warehouse-orders-data.csv) and records are saved in the format: order_id,customer_id,warehouse_id,order_date,shipper_date

Below is a preview of both tables in .csv format:

![Data in csv format](c05m03-warehouse-tables-data.png 'Data in csv format')

## Importing the data in BigQuery

The following steps are followed to import the employees and departments data to BigQuery:

- **Create dataset** with **Dataset ID** `warehouse_orders`
- In the **Dataset info** window, select the **CREATE TABLE** button
- In the **Source** section, select the ***Upload*** option in **Create table from**
- Browse to the `c05m03-warehouse-data.csv` file and open
- Set the file format to `.csv`
- In the **Destination** section, name the table as `warehouse`
- In the **Schema** section, select **Auto detect**
- Finally, select **Create table**

A new table `warehouse` has been created and appear in the explorer pane under the database `warehouse_orders`. The above steps are repeated to create a new table `orders` from the file `c05m03-warehouse-orders-data.csv`. A preview of the BigQuery tables are shown below:

![Data in BigQuery](c05m03-warehouse-tables-bigquery.png 'Data in BigQuery')

## Query: How many states have warehouses that shipped orders?

I execute the following query to see if COUNT will answer the question:

In [ ]:
SELECT
	COUNT(warehouse.state) as num_states
FROM
    `plucky-aegis-427011-v5.warehouse_orders.orders` AS orders
JOIN
    `plucky-aegis-427011-v5.warehouse_orders.warehouse` warehouse
ON orders.warehouse_id = warehouse.warehouse_id;

The output of the query is 9999 for num_states. This is clearly not the answer I was looking for as it is counting every instance of the warehouse state, which will be once per order and thus giving me the total number of orders. Instead, I will adjust the query to include DISTINCT to remove the duplicates and identify the number of unique states as follows:

In [ ]:
SELECT
	COUNT(DISTINCT warehouse.state) as num_states
FROM
	`plucky-aegis-427011-v5.warehouse_orders.orders` AS orders
JOIN
  `plucky-aegis-427011-v5.warehouse_orders.warehouse` warehouse
ON orders.warehouse_id = warehouse.warehouse_id;

The query has identified that we have shipped orders from warehouses located in 3 different states, which is the answer I was looking for. Next I will adjust the query so that I can see the states and number of orders shipped out of each. I execute the following query:

In [ ]:
SELECT
	warehouse.state,
	--Total number of distinct orders
    COUNT(DISTINCT orders.order_id) as num_orders
FROM
	`plucky-aegis-427011-v5.warehouse_orders.orders` AS orders
JOIN
  `plucky-aegis-427011-v5.warehouse_orders.warehouse` warehouse
ON orders.warehouse_id = warehouse.warehouse_id
GROUP BY
	warehouse.state;

The output I receive summarize the 3 state along with the number of orders placed in each state that make up the 9999 orders. Below is a preview of the output:

![Number of orders by state](c05m03-query-orders-by-state.png 'Number of orders by state')

## Query: Which warehouses and states are shipping the most orders?

To answer this question, I execute the following query sorting the output by number of orders in descending order:

In [ ]:
SELECT
    warehouse.warehouse_alias,
    warehouse.state,
    --Total number of distinct orders
    COUNT(DISTINCT orders.order_id) as num_orders
FROM
	`plucky-aegis-427011-v5.warehouse_orders.orders` AS orders
JOIN
    `plucky-aegis-427011-v5.warehouse_orders.warehouse` warehouse
ON orders.warehouse_id = warehouse.warehouse_id
GROUP BY
	warehouse.warehouse_alias,
	warehouse.state
--Sort from most orders to least
ORDER BY
	num_orders DESC;

The Lansing Fulfillment Center ships the most with 3178 orders shipped. Both warehouse in MI (Michigan) are at position 1 and 2, respectively, and is the state shipping the most orders with a total order number of 6205. Below is a preview of the query output:

![Number of orders by warehouse](c05m03-query-warehouse-orders.png 'Number of orders by warehouse')